# Question: How did the revenue of Restaurant 1 and Restaurant 2 change over time?

In [0]:
sqlContext
import pyspark.sql.functions as f

### Loading Orders data for both Restaurants

In [0]:
restaurant1 = sqlContext.read.load('/FileStore/tables/restaurant_1_orders-2.csv', format='csv', header=True)

restaurant_2 = sqlContext.read.load('/FileStore/tables/restaurant_2_orders-2.csv', format='csv', header=True)

In [0]:
restaurant1.show(5)
restaurant_2.show(5)

+------------+----------------+-------------------+--------+-------------+--------------+
|Order Number|      Order Date|          Item Name|Quantity|Product Price|Total products|
+------------+----------------+-------------------+--------+-------------+--------------+
|       16118|03/08/2019 20:25|      Plain Papadum|       2|          0.8|             6|
|       16118|03/08/2019 20:25|   King Prawn Balti|       1|        12.95|             6|
|       16118|03/08/2019 20:25|        Garlic Naan|       1|         2.95|             6|
|       16118|03/08/2019 20:25|      Mushroom Rice|       1|         3.95|             6|
|       16118|03/08/2019 20:25|Paneer Tikka Masala|       1|         8.95|             6|
+------------+----------------+-------------------+--------+-------------+--------------+
only showing top 5 rows

+--------+----------------+--------------------+--------+-------------+--------------+
|Order ID|      Order Date|           Item Name|Quantity|Product Price|Total p

In [0]:
restaurant1.dtypes

Out[442]: [('Order Number', 'string'),
 ('Order Date', 'string'),
 ('Item Name', 'string'),
 ('Quantity', 'string'),
 ('Product Price', 'string'),
 ('Total products', 'string')]

### We will make the following changes to the dataframes:
 1. Change the header names in accordance to [PEP 8](https://peps.python.org/pep-0008/).
 2. Drop 'Item Name' and 'Total Products'.
 3. Add a 'Total Price' column, which is 'Product Price' * 'Quantity'.
 4. Group orders by 'Order Date' and show the total value of each order.

In [0]:
#1
new_column_names = ['order_number', 'order_date', 'item_name', 'quantity', 'product_price', 'total_products']

restaurant1 = restaurant1.toDF(*new_column_names)
restaurant_2 = restaurant_2.toDF(*new_column_names)

In [0]:
#2
restaurant1 = restaurant1.drop('item_name', 'total_products')
restaurant_2 = restaurant_2.drop('item_name', 'total_products')

In [0]:
restaurant1.show(5)
restaurant_2.show(5)

+------------+----------------+--------+-------------+
|order_number|      order_date|quantity|product_price|
+------------+----------------+--------+-------------+
|       16118|03/08/2019 20:25|       2|          0.8|
|       16118|03/08/2019 20:25|       1|        12.95|
|       16118|03/08/2019 20:25|       1|         2.95|
|       16118|03/08/2019 20:25|       1|         3.95|
|       16118|03/08/2019 20:25|       1|         8.95|
+------------+----------------+--------+-------------+
only showing top 5 rows

+------------+----------------+--------+-------------+
|order_number|      order_date|quantity|product_price|
+------------+----------------+--------+-------------+
|       25583|03/08/2019 21:58|       1|        11.95|
|       25583|03/08/2019 21:58|       1|         3.95|
|       25583|03/08/2019 21:58|       2|         3.95|
|       25583|03/08/2019 21:58|       1|         2.95|
|       25583|03/08/2019 21:58|       1|         2.95|
+------------+----------------+--------+

In [0]:
#2
restaurant1 = restaurant1.withColumn('total_price', f.col('quantity') * f.col('product_price'))
restaurant_2 = restaurant_2.withColumn('total_price', f.col('quantity') * f.col('product_price'))

In [0]:
restaurant1.show(3)
restaurant_2.show(3)


+------------+----------------+--------+-------------+-----------+
|order_number|      order_date|quantity|product_price|total_price|
+------------+----------------+--------+-------------+-----------+
|       16118|03/08/2019 20:25|       2|          0.8|        1.6|
|       16118|03/08/2019 20:25|       1|        12.95|      12.95|
|       16118|03/08/2019 20:25|       1|         2.95|       2.95|
+------------+----------------+--------+-------------+-----------+
only showing top 3 rows

+------------+----------------+--------+-------------+-----------+
|order_number|      order_date|quantity|product_price|total_price|
+------------+----------------+--------+-------------+-----------+
|       25583|03/08/2019 21:58|       1|        11.95|      11.95|
|       25583|03/08/2019 21:58|       1|         3.95|       3.95|
|       25583|03/08/2019 21:58|       2|         3.95|        7.9|
+------------+----------------+--------+-------------+-----------+
only showing top 3 rows



In [0]:
#4
restaurant1 = restaurant1.groupBy('order_date').sum('total_price').withColumnRenamed('sum(total_price)', 'order_total')

restaurant_2 = restaurant_2.groupBy('order_date').sum('total_price').withColumnRenamed('sum(total_price)', 'order_total')

In [0]:
restaurant1.show(3)
restaurant_2.show(3)

+----------------+------------------+
|      order_date|       order_total|
+----------------+------------------+
|14/06/2019 19:14|             22.45|
|22/04/2019 18:35|             40.95|
|16/02/2019 18:45|118.55000000000003|
+----------------+------------------+
only showing top 3 rows

+----------------+------------------+
|      order_date|       order_total|
+----------------+------------------+
|07/04/2019 20:33|             28.25|
|03/04/2019 18:55|52.650000000000006|
|23/02/2019 19:10|              42.0|
+----------------+------------------+
only showing top 3 rows



Adding a year column:

In [0]:
restaurant1 = restaurant1.withColumn('year', restaurant1.order_date.substr(7, 4))
restaurant_2 = restaurant_2.withColumn('year', restaurant_2.order_date.substr(7,4))

Group by year

In [0]:
restaurant1 = restaurant1.groupBy('year').sum('order_total').withColumnRenamed('sum(order_total)', 'yearly_order_total')

restaurant_2 = restaurant_2.groupBy('year').sum('order_total').withColumnRenamed('sum(order_total)', 'yearly_order_total')

Rounding the 'yearly_order_total' values and ordering by total:

In [0]:
restaurant1 = restaurant1.select("*", f.round('yearly_order_total', 2)).withColumnRenamed('round(yearly_order_total, 2)', 'yearly_revenue').drop('yearly_order_total').orderBy('yearly_revenue')

restaurant_2 = restaurant_2.select("*", f.round('yearly_order_total', 2)).withColumnRenamed('round(yearly_order_total, 2)', 'yearly_revenue').drop('yearly_order_total').orderBy('yearly_revenue')

In [0]:
display(restaurant1)
display(restaurant_2)

year,yearly_revenue
2015,412.5
2016,63978.8
2019,105546.95
2017,126484.5
2018,150094.7


year,yearly_revenue
2015,32.65
2016,89834.7
2019,150403.2
2017,209038.1
2018,237790.2


### Conclusion

- Both restaurants seem to have increasing revenue from 2015 to 2019 overall.
- The revenues increased for 4 consecutive years before decreasing by 29.7% and 36.7% for restaurant 1 and 2 respectively.